In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
from PIL import Image, ImageOps, ImageFilter
import cv2
import numpy as np
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity

# Приготовление датасета

In [ ]:
"""Функция перевода изображений из тензоров в PIL для инференса"""

def tensor_to_pil(tensor_img):
    unnorm = transforms.Normalize(mean=[-0.5/0.5]*3, std=[1/0.5]*3)
    tensor_img = unnorm(tensor_img).clamp(0, 1)
    pil_img = transforms.ToPILImage()(tensor_img)
    return pil_img

"""Выгрузка изображений в Dataset"""

class PairedImageDataset(Dataset):
    def __init__(self, dataset_root, image_size=512):
        self.pairs = []
        cover_root = os.path.join(dataset_root, "cover")
        final_root = os.path.join(dataset_root, "final")
        for subfolder in sorted(os.listdir(cover_root)):
            cover_sub = os.path.join(cover_root, subfolder)
            final_sub = os.path.join(final_root, subfolder)
            if os.path.isdir(cover_sub) and os.path.isdir(final_sub):
                cover_files = sorted([f for f in os.listdir(cover_sub) if f.lower().endswith(".png")])
                for fname in cover_files:
                    cover_path = os.path.join(cover_sub, fname)
                    final_fname = os.path.splitext(fname)[0] + ".jpg"
                    final_path = os.path.join(final_sub, final_fname)
                    if not os.path.exists(final_path):
                        final_path = os.path.join(final_sub, fname)
                    if os.path.exists(final_path):
                        self.pairs.append((cover_path, final_path))
                    else:
                        print(f"Warning: No matching final image for {cover_path}")
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3),
        ])
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        cover_path, final_path = self.pairs[idx]
        cover_img = Image.open(cover_path).convert("RGB")
        final_img = Image.open(final_path).convert("RGB")
        return {
            "cover": self.transform(cover_img),
            "final": self.transform(final_img)
        }


# Функции для инференса

In [ ]:
# -------------------------
# Вспомогательные функции
# -------------------------

def add_gaussian_noise(image, mean=0, std=2):
    """Добавляет слабый гауссов шум к изображению PIL с обрезкой значений."""
    img_np = np.array(image)  # Преобразуем изображение в массив numpy
    noise = np.random.normal(mean, std, img_np.shape).astype(np.float32)  # Генерируем шум
    noisy_img = img_np.astype(np.float32) + noise  # Добавляем шум
    noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)  # Ограничиваем значения в допустимом диапазоне
    return Image.fromarray(noisy_img)  # Конвертируем обратно в PIL Image

def apply_perspective_transform(image, distortion_scale=0.1):
    """Применяет случайное перспективное искажение к изображению PIL."""
    img_np = np.array(image)
    h, w = img_np.shape[:2]  # Получаем размеры изображения
    
    # Исходные угловые точки изображения
    src_points = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    
    # Случайно смещаем углы изображения
    dst_points = src_points + np.random.uniform(-distortion_scale, distortion_scale, size=(4, 2)) * [w, h]
    dst_points = dst_points.astype(np.float32)  # Убедимся, что тип данных float32
    
    # Вычисляем матрицу перспективного преобразования
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Применяем перспективное преобразование
    warped = cv2.warpPerspective(img_np, M, (w, h))
    
    return Image.fromarray(warped)  # Возвращаем преобразованное изображение

def apply_basic_transform(image, max_angle=10, max_pad=0.1, noise_std=2):
    """Применяет базовые аугментации к изображению PIL."""
    # Случайный поворот с темно-серым фоном
    angle = random.uniform(-max_angle, max_angle)
    rotated = image.rotate(angle, resample=Image.BICUBIC, expand=True, fillcolor=(10, 10, 10))
    
    # Небольшое размытие для сглаживания краев
    rotated = rotated.filter(ImageFilter.GaussianBlur(radius=0.5))
    
    # Добавление случайного паддинга с темно-серым цветом
    w, h = rotated.size
    pad_left = int(random.uniform(0, max_pad) * w)
    pad_right = int(random.uniform(0, max_pad) * w)
    pad_top = int(random.uniform(0, max_pad) * h)
    pad_bottom = int(random.uniform(0, max_pad) * h)
    padded = ImageOps.expand(rotated, border=(pad_left, pad_top, pad_right, pad_bottom), fill=(10, 10, 10))
    
    # Добавление небольшого шума
    if noise_std > 0:
        padded = add_gaussian_noise(padded, std=noise_std)
    
    return padded

def apply_advanced_transform(image, max_angle=3, max_pad=0.1, distortion_scale=0.1, noise_std=0.2):
    """Применяет сложные аугментации к изображению PIL."""
    # Перспективное искажение
    persp = apply_perspective_transform(image, distortion_scale)
    
    # Случайный поворот с темно-серым фоном
    angle = random.uniform(-max_angle, max_angle)
    rotated = persp.rotate(angle, resample=Image.BICUBIC, expand=True, fillcolor=(10, 10, 10))
    
    # Размытие для сглаживания краев
    rotated = rotated.filter(ImageFilter.GaussianBlur(radius=0.5))
    
    # Добавление случайного паддинга с темно-серым фоном
    w, h = rotated.size
    pad_left = int(random.uniform(0, max_pad) * w)
    pad_right = int(random.uniform(0, max_pad) * w)
    pad_top = int(random.uniform(0, max_pad) * h)
    pad_bottom = int(random.uniform(0, max_pad) * h)
    padded = ImageOps.expand(rotated, border=(pad_left, pad_top, pad_right, pad_bottom), fill=(10, 10, 10))
    
    # Добавление небольшого шума
    if noise_std > 0:
        padded = add_gaussian_noise(padded, std=noise_std)
    
    return padded

def compute_metrics(img_target, img_generated):
    """
    Вычисляет MSE, PSNR и SSIM между двумя изображениями PIL одинакового размера.
    """
    # Преобразуем изображения в numpy-массивы float32 с нормализацией в диапазоне [0,1]
    np_target = np.array(img_target).astype(np.float32) / 255.0
    np_generated = np.array(img_generated).astype(np.float32) / 255.0

    # 1) Среднеквадратичная ошибка (MSE)
    mse_val = mean_squared_error(np_target, np_generated)

    # 2) Пиковое отношение сигнал/шум (PSNR)
    psnr_val = peak_signal_noise_ratio(np_target, np_generated, data_range=1.0)

    # 3) Структурное сходство (SSIM)
    h, w, c = np_target.shape
    max_win_size = min(h, w, 7)  # Если минимальное измерение < 7, уменьшаем размер окна
    ssim_val = structural_similarity(
        np_target, 
        np_generated,
        data_range=1.0,
        channel_axis=-1,
        win_size=max_win_size
    )

    return mse_val, psnr_val, ssim_val

# Архитектура модели

In [ ]:
class STN(nn.Module):
    def __init__(self):
        super(STN, self).__init__()
        self.localization = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=7),  # Извлечение признаков из входного изображения
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.ReLU(True),
            nn.AdaptiveAvgPool2d((2, 2))  # Сведение карты активаций к фиксированному размеру
        )
        self.fc = nn.Sequential(
            nn.Linear(10 * 2 * 2, 32),  # Полносвязные слои для вычисления параметров аффинного преобразования
            nn.ReLU(True),
            nn.Linear(32, 6)
        )
        self.fc[2].weight.data.zero_()  # Инициализация весов последнего слоя нулями
        self.fc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))  # Инициализация биаса единичной матрицей
        
    def forward(self, x):
        batch_size = x.size(0)
        xs = self.localization(x)
        xs = xs.view(batch_size, -1)  # Приведение к векторному виду
        theta = self.fc(xs)
        theta = theta.view(-1, 2, 3)  # Преобразование в матрицу аффинного преобразования
        grid = F.affine_grid(theta, x.size(), align_corners=True)
        x_transformed = F.grid_sample(x, grid, align_corners=True)  # Применение преобразования к входному изображению
        return x_transformed, theta

class HDRNet(nn.Module):
    def __init__(self, input_nc=3, output_nc=3, grid_size=16):
        super(HDRNet, self).__init__()
        self.grid_size = grid_size  # Размер сетки трансформации (глобальной коррекции)
        
        # Локальная ветка (Local branch): предсказывает низкоразрешенную карту преобразования
        # Использует дилатационные свертки для расширения области восприятия и извлечения дополнительных деталей
        self.local_branch = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=3, stride=1, padding=1),  # Базовый сверточный слой
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=2, dilation=2),  # Дилатационная свертка для большего охвата контекста
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Дополнительный слой для уточнения признаков
            nn.ReLU(inplace=True),
            nn.Conv2d(64, output_nc, kernel_size=3, stride=1, padding=1),  # Предсказание параметров преобразования
            nn.AdaptiveAvgPool2d((grid_size, grid_size))  # Приведение карты преобразования к низкому разрешению
        )
        
        # Глобальная ветка (Global branch): вычисляет общую коррекцию цвета/тона
        self.global_branch = nn.Sequential(
            nn.Conv2d(input_nc, 32, kernel_size=3, stride=2, padding=1),  # Первый сверточный слой с уменьшением размера в 2 раза
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Второй сверточный слой, еще раз уменьшающий размер
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)  # Приведение к размеру (B, 64, 1, 1) - общий вектор признаков
        )
        self.global_fc = nn.Linear(64, output_nc)  # Полносвязный слой для предсказания общей коррекции

    def forward(self, x):
        # Локальная ветка: предсказывает сетку трансформации и увеличивает ее до размера входного изображения
        local_grid = self.local_branch(x)  # Выходная форма: (B, output_nc, grid_size, grid_size)
        local_adjust = nn.functional.interpolate(local_grid, size=x.shape[2:], mode='bilinear', align_corners=True)

        # Глобальная ветка: вычисляет глобальную коррекцию
        batch_size = x.size(0)
        global_feat = self.global_branch(x)  # Выходная форма: (B, 64, 1, 1)
        global_feat = global_feat.view(batch_size, -1)  # Преобразуем вектор в (B, 64)
        global_adjust = self.global_fc(global_feat)  # Выходная форма: (B, output_nc)
        global_adjust = global_adjust.unsqueeze(2).unsqueeze(3).expand_as(local_adjust)  # Расширение до размера локальной коррекции

        # Объединяем локальную и глобальную коррекцию
        adjustment = local_adjust + global_adjust
        
        # Остаточное соединение (Residual connection): добавляем рассчитанную коррекцию к входному изображению
        out = x + adjustment
        
        # Применяем активацию tanh для нормализации выходных значений в диапазоне [-1, 1]
        return torch.tanh(out)



# Многоуровневый дискриминатор (Multi-Scale Discriminator)
class NLayerDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3):
        super(NLayerDiscriminator, self).__init__()
        kw = 4  # Размер ядра свертки
        padw = 1  # Паддинг
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),  # Первая сверточная операция с уменьшением размера в 2 раза
            nn.LeakyReLU(0.2, True)  # Активация с утечкой, предотвращает исчезновение градиента
        ]
        self.layers = nn.ModuleList([nn.Sequential(*sequence)])
        nf_mult = 1  # Коэффициент увеличения количества каналов

        # Добавляем сверточные слои с Instance Normalization
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)  # Увеличиваем количество фильтров, но не больше 8 раз
            self.layers.append(nn.Sequential(
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw),
                nn.InstanceNorm2d(ndf * nf_mult),  # Нормализация для стабилизации обучения
                nn.LeakyReLU(0.2, True)
            ))

        # Дополнительный слой без изменения размера
        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        self.layers.append(nn.Sequential(
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw),
            nn.InstanceNorm2d(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ))

        # Финальный слой, который выдает одно значение для каждой области изображения
        self.output_layer = nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)
        
    def forward(self, input):
        result = []
        x = input
        for layer in self.layers:
            x = layer(x)
            result.append(x)  # Сохраняем промежуточные карты активаций
        out = self.output_layer(x)
        result.append(out)  # Добавляем финальный выход
        return result  # Возвращаем список промежуточных карт признаков и финальный выход


# Многоуровневый дискриминатор, использующий несколько NLayerDiscriminator на разных масштабах изображения
class MultiScaleDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3, num_D=3):
        super(MultiScaleDiscriminator, self).__init__()
        self.num_D = num_D  # Количество дискриминаторов
        self.discriminators = nn.ModuleList()

        # Создаем num_D экземпляров NLayerDiscriminator
        for _ in range(num_D):
            self.discriminators.append(NLayerDiscriminator(input_nc, ndf, n_layers))

        # Среднее сглаживание (downsampling) для понижения разрешения изображения
        self.downsample = nn.AvgPool2d(3, stride=2, padding=[1, 1], count_include_pad=False)
        
    def forward(self, input):
        results = []
        for D in self.discriminators:
            results.append(D(input))  # Обрабатываем вход через дискриминатор
            input = self.downsample(input)  # Уменьшаем размер изображения перед подачей в следующий дискриминатор
        return results  # Возвращаем выходы всех дискриминаторов

# Обучение модели

Параметры

In [ ]:
num_epochs = 200
batch_size = 4
lr = 1e-4
lr_G = lr
lambda_l1 = 2.0      # Вес L1-ошибки (реконструкция)
lambda_feat = 10.0   # Вес Feature Matching Loss

dataset_root = "/kaggle/input/screen2photo-dataset-augm-and-sep/augment_train_data/kaggle/working/augment_train_data"
kaggle_output_dir = "/kaggle/working"

checkpoint_interval = 10  # Интервал сохранения чекпоинтов
#Путь до модели в формате .pth. Если обучение с нуля, то выставьте resume_checkpoint = None
resume_checkpoint = '/kaggle/input/pseudo_hdrnetstn-ver-final-ep-20/pytorch/default/1/checkpoint_hdrnet_v2_epoch_20.pth'
start_epoch = 20 #с какой эпохи стартует чекпоинт (если с нуля, то выставьте 0)

In [ ]:
img_size_for_model = 256  #не менять
train_dataset = PairedImageDataset(dataset_root, image_size=img_size_for_model)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# -------------------------
# Функции загрузки/сохранения чекпоинтов
# -------------------------
def load_checkpoint(checkpoint_path, generator, discriminator, stn_module, optimizer_G, optimizer_D):
    print(f"Загружаем чекпоинт {checkpoint_path} ...")
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    
    generator.load_state_dict(checkpoint["generator"])
    discriminator.load_state_dict(checkpoint["discriminator"])
    optimizer_G.load_state_dict(checkpoint["optimizer_G"])
    optimizer_D.load_state_dict(checkpoint["optimizer_D"])
    
    start_epoch = checkpoint["epoch"]
    print(f"Чекпоинт загружен. Продолжаем обучение с {start_epoch} эпохи.\n")
    return start_epoch

def save_checkpoint(checkpoint_path, epoch, generator, discriminator, stn_module, optimizer_G, optimizer_D):
    print(f"Сохраняем чекпоинт в {checkpoint_path} ...")
    checkpoint = {
        "epoch": epoch,
        "generator": generator.state_dict(),
        "discriminator": discriminator.state_dict(),
        "stn_module": stn_module.state_dict(),
        "optimizer_G": optimizer_G.state_dict(),
        "optimizer_D": optimizer_D.state_dict()
    }
    torch.save(checkpoint, checkpoint_path)
    print("Чекпоинт сохранен.\n")

# -------------------------
# Настройки устройства
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используется устройство:", device)

# Инициализация моделей
generator = HDRNet(input_nc=3, output_nc=3).to(device)
stn_module = STN().to(device)

# Многоуровневый дискриминатор (Multi-Scale Discriminator)
discriminator = MultiScaleDiscriminator(input_nc=6, ndf=64, n_layers=3, num_D=3).to(device)

# Функции потерь
criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

# Оптимизаторы (учим и HDRNet, и STN)
optimizer_G = optim.Adam(list(generator.parameters()) + list(stn_module.parameters()), lr=lr_G, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# -------------------------
# Загрузка чекпоинта, если продолжаем обучение
# -------------------------
if resume_checkpoint is not None and os.path.exists(resume_checkpoint):
    start_epoch = load_checkpoint(resume_checkpoint, generator, discriminator, stn_module, optimizer_G, optimizer_D)

# Замораживание STN после определенной эпохи
freeze_stn_epoch = 200
if start_epoch >= freeze_stn_epoch:
    for param in stn_module.parameters():
        param.requires_grad = False
    print(f"STN заморожен (обучение возобновлено после {freeze_stn_epoch} эпохи)")


In [ ]:
for epoch in range(start_epoch, num_epochs):
    generator.train()
    stn_module.train()
    discriminator.train()
    
    # Замораживание STN после определенной эпохи
    if epoch >= freeze_stn_epoch:
        for param in stn_module.parameters():
            param.requires_grad = False

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in progress_bar:
        real_A = batch["cover"].to(device)   # Входное изображение (экран)
        real_B = batch["final"].to(device)   # Целевое изображение (фото экрана)

        # STN исправляет перспективные искажения в real_B
        fixed_real_B, theta_real = stn_module(real_B)

        # Генерация изображения из real_A с помощью HDRNet
        fake_B = generator(real_A)

        # Подготовка данных для дискриминатора (объединяем вход и целевое изображение)
        real_AB = torch.cat([real_A, fixed_real_B], dim=1)  # Реальная пара
        fake_AB = torch.cat([real_A, fake_B], dim=1)        # Фейковая пара

        # -----------------
        # Обучение генератора
        # -----------------
        optimizer_G.zero_grad()

        # GAN-ошибка (насколько хорошо фейковое изображение обманывает дискриминатор)
        pred_fake = discriminator(fake_AB)
        with torch.no_grad():
            pred_real = discriminator(real_AB)

        loss_GAN = 0.0
        for scale_out in pred_fake:
            out = scale_out[-1]
            valid_ = torch.ones_like(out, device=device)
            loss_GAN += criterion_GAN(out, valid_)
        loss_GAN /= discriminator.num_D

        # Ошибка сопоставления признаков (Feature Matching Loss)
        loss_FM = 0.0
        for i in range(discriminator.num_D):
            num_intermediate = len(pred_fake[i]) - 1
            for j in range(num_intermediate):
                loss_FM += criterion_L1(pred_fake[i][j], pred_real[i][j])
        loss_FM /= (discriminator.num_D * num_intermediate)

        # L1-ошибка между сгенерированным изображением и исправленным реальным изображением
        loss_L1_ = criterion_L1(fake_B, fixed_real_B)

        # Итоговая функция ошибки генератора
        loss_G = loss_GAN + lambda_l1 * loss_L1_ + lambda_feat * loss_FM
        loss_G.backward()
        optimizer_G.step()

        # ---------------------
        # Обучение дискриминатора
        # ---------------------
        optimizer_D.zero_grad()
        pred_fake = discriminator(fake_AB.detach())  # Отключаем градиенты для генератора

        loss_D = 0.0
        for i in range(discriminator.num_D):
            out_real = pred_real[i][-1]
            out_fake = pred_fake[i][-1]
            valid_ = torch.ones_like(out_real, device=device)
            fake_ = torch.zeros_like(out_fake, device=device)
            loss_real = criterion_GAN(out_real, valid_)
            loss_fake = criterion_GAN(out_fake, fake_)
            loss_D += 0.5 * (loss_real + loss_fake)
        loss_D /= discriminator.num_D
        loss_D.backward()
        optimizer_D.step()

        # Отображение текущих ошибок
        progress_bar.set_postfix({
            "loss_G": loss_G.item(),
            "loss_D": loss_D.item(),
            "L1": loss_L1_.item(),
            "FM": loss_FM.item()
        })

    # Вывод промежуточных результатов каждые 5 эпох
    if (epoch + 1) % 5 == 0:
        print(f"[INFO] Epoch {epoch+1}/{num_epochs} completed. Loss_G: {loss_G.item():.4f}, Loss_D: {loss_D.item():.4f}")

    # Сохранение чекпоинта каждые checkpoint_interval эпох
    if (epoch + 1) % checkpoint_interval == 0:
        ckpt_path = os.path.join(kaggle_output_dir, f"checkpoint_epoch_{epoch+1}.pth")
        save_checkpoint(ckpt_path, epoch+1, generator, discriminator, stn_module, optimizer_G, optimizer_D)

# Финальное сохранение модели
final_ckpt_path = os.path.join(kaggle_output_dir, f"checkpoint_epoch_{num_epochs}.pth")
save_checkpoint(final_ckpt_path, num_epochs, generator, discriminator, stn_module, optimizer_G, optimizer_D)
print(f"Обучение завершено. Финальный чекпоинт сохранен в {final_ckpt_path}")

# Инференс

In [ ]:
advanced_mod = True
# Путь к датасету для инференса
inference_dataset_root = "/kaggle/input/screen2photo-dataset-augm-and-sep/inference_data/kaggle/working/inference_data"

use_uploaded_model = True  
# Путь к чекпоинту для инференса
uploaded_checkpoint_path = "/kaggle/input/111/pytorch/default/1/checkpoint_epoch_30 (1).pth"

In [ ]:
inference_full_dataset = PairedImageDataset(inference_dataset_root, image_size=img_size_for_model)
print(f"Загружен инференс-датасет с {len(inference_full_dataset)} парами изображений.")

# Выбор функции трансформации (простая или продвинутая)
if advanced_mod:
    transform_func = apply_advanced_transform
else:
    transform_func = apply_basic_transform  # Можно переключить на advanced, если нужно


# -------------------------
# Загрузка предобученной модели
# -------------------------

if use_uploaded_model:
    ckpt = torch.load(uploaded_checkpoint_path, map_location=device, weights_only=True)
    generator.load_state_dict(ckpt["generator"])  # Загружаем только веса генератора
    print(f"Загружены веса генератора из: {uploaded_checkpoint_path}")

generator.eval()  # Переводим генератор в режим инференса

# -------------------------
# Инференс и вычисление метрик
# -------------------------

mse_list, psnr_list, ssim_list = [], [], []

for idx in range(len(inference_full_dataset)):
    sample = inference_full_dataset[idx]
    cover_tensor = sample["cover"].to(device)  # Входное изображение (экран)
    final_tensor = sample["final"].to(device)  # Целевое изображение (фото экрана)

    with torch.no_grad():
        fake_tensor = generator(cover_tensor.unsqueeze(0)).squeeze(0)  # Генерируем изображение

    # Преобразуем тензор в изображение PIL
    fake_image = tensor_to_pil(fake_tensor.cpu())

    # Применяем выбранную трансформацию
    transformed_image = transform_func(
        fake_image,
        max_angle=3,
        max_pad=0.01,
        distortion_scale=0.05,
        noise_std=0.1
    )

    # Преобразуем целевое изображение в PIL
    final_image = tensor_to_pil(final_tensor.cpu())

    # Приводим целевое изображение к размеру трансформированного
    w, h = transformed_image.size
    final_resized = final_image.resize((w, h), Image.Resampling.BILINEAR)

    # Вычисляем метрики
    mse_val, psnr_val, ssim_val = compute_metrics(final_resized, transformed_image)
    mse_list.append(mse_val)
    psnr_list.append(psnr_val)
    ssim_list.append(ssim_val)

    # Визуализация изображений (по желанию)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(tensor_to_pil(cover_tensor.cpu()))
    axes[0].set_title("Cover (Input)")
    axes[0].axis("off")

    axes[1].imshow(transformed_image)
    axes[1].set_title("Transformed (Generated)")
    axes[1].axis("off")

    axes[2].imshow(final_image)
    axes[2].set_title("Final (Target)")
    axes[2].axis("off")

    plt.tight_layout()
    plt.show()

# -------------------------
# Вывод средних метрик по всему датасету
# -------------------------

N = len(mse_list)
avg_mse = sum(mse_list) / N
avg_psnr = sum(psnr_list) / N
avg_ssim = sum(ssim_list) / N

print(f"Среднее MSE : {avg_mse:.4f}")
print(f"Среднее PSNR: {avg_psnr:.2f} dB")
print(f"Среднее SSIM: {avg_ssim:.4f}")